# Neural Machine Translation: Building an RNN Encoder-Decoder with Keras 3

**Learning Objectives**
1.  Learn how to build an efficient `tf.data.Dataset` pipeline for a seq2seq task.
2.  Learn how to preprocess text using the `keras.layers.TextVectorization` layer.
3.  Learn how to train an encoder-decoder model in Keras using the Functional API.
4.  Learn how to create separate encoder and decoder models for inference.
5.  Learn how to implement a translation (decoding) function from scratch.
6.  Learn how to use the BLEU score to evaluate a translation model.


***Note: For faster execution, please ensure you are using a GPU runtime. An NVIDIA T4 GPU is recommended for this notebook.***

## Introduction

In this notebook, we'll build a Spanish-to-English translation model using a modern RNN encoder-decoder architecture with Keras 3.

We will start by building an efficient and scalable input pipeline with the `tf.data.Dataset` API. A key part of our workflow will be using the `TextVectorization` layer to handle all text preprocessing—from standardization and tokenization to integer-encoding—directly within our model.

Next, we will use the Keras Functional API to build and train our RNN encoder-decoder model. After training, we will create two specialized models—a dedicated encoder and a decoder—from the layers of our trained model. These specialized models are essential for performing inference, allowing us to implement a function that generates translations word by word.

Finally, we'll evaluate the quality of our model's translations using the industry-standard BLEU score.


In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import pickle
import re
import string
import sys

import evaluate
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import GRU, Dense, Embedding, Input
from keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from tqdm import tqdm

print(tf.__version__)

2025-10-15 02:22:52.430403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760494972.451957   31154 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760494972.458525   31154 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.18.1


In [3]:
SEED = 42
MODEL_PATH = "translate_models/baseline"
DATA_URL = (
    "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
)
LOAD_CHECKPOINT = False

In [4]:
keras.utils.set_random_seed(SEED)

## Downloading Data

We'll use a language dataset provided by http://www.manythings.org/anki/. The dataset contains Spanish-English  translation pairs in the format:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

The dataset is a curated list of 120K translation pairs from http://tatoeba.org/, a platform for community contributed translations by native speakers.

In [5]:
path_to_zip = keras.utils.get_file("spa-eng.zip", origin=DATA_URL, extract=True)

path_to_file = os.path.join(
    os.path.dirname(path_to_zip), "spa-eng_extracted/spa-eng/spa.txt"
)
print("Translation data stored at:", path_to_file)

Translation data stored at: /home/jupyter/.keras/datasets/spa-eng_extracted/spa-eng/spa.txt


In [6]:
data = pd.read_csv(
    path_to_file, sep="\t", header=None, names=["english", "spanish"]
)

In [7]:
data.sample(3)

,english,spanish
1025,How boring!,¡Qué aburrimiento!
4338,I love sports.,Adoro el deporte.
55586,Would you like to swap jobs?,¿Te gustaría que intercambiemos los trabajos?


## Create the tf.data Pipeline
To begin, we'll construct our training and evaluation datasets using the `tf.data` API, which is the standard for building efficient and scalable input pipelines in TensorFlow. This approach allows us to handle data in a memory-efficient way and seamlessly integrate it with Keras.

Our process will be as follows:

1. Load the data: We create a tf.data.Dataset directly from our pandas DataFrame using tf.data.Dataset.from_tensor_slices. This creates a dataset where each element is a pair of (Spanish, English) sentences.

2. Split the data: We'll use the .take() and .skip() methods to create our training and validation sets. This is a clean and efficient way to split the data without having to load everything into memory at once.

3. Define a standardization function: We create a custom_standardization function to preprocess our raw text. This function replicates the logic of the original preprocess_sentence function by lowercasing text, adding spaces around punctuation, and, most importantly, adding <start> and <end> tokens to each sentence. It is built using tf.strings operations, which allows it to be embedded directly into our TextVectorization layer and run efficiently on the GPU/TPU.

In [8]:
# Define constants
BUFFER_SIZE = 32000
BATCH_SIZE = 64
TEST_PROP = 0.2
NUM_EXAMPLES = 30000

# Create a single dataset from your pandas DataFrame
full_dataset = tf.data.Dataset.from_tensor_slices(
    (data["spanish"][:NUM_EXAMPLES], data["english"][:NUM_EXAMPLES])
)

# Create the training and validation splits using take() and skip()
TRAIN_SIZE = int(NUM_EXAMPLES * (1 - TEST_PROP))
train_raw = full_dataset.take(TRAIN_SIZE)
val_raw = full_dataset.skip(TRAIN_SIZE)

# Define the custom standardization function for TextVectorization


@tf.keras.utils.register_keras_serializable()
def custom_standardization(input_string):
    """Replicates the original preprocess_sentence function using tf.strings."""
    # Lowercase and strip leading/trailing whitespace
    s = tf.strings.lower(input_string)
    s = tf.strings.strip(s)

    # Add spaces around punctuation
    s = tf.strings.regex_replace(s, r"([?.!,¿])", r" \1 ")

    # Replace multiple spaces with a single space
    s = tf.strings.regex_replace(s, r'[" "]+', " ")

    # Filter out unwanted characters, replacing them with a space
    s = tf.strings.regex_replace(s, r"[^a-zA-Z?.!,¿]+", " ")

    # Strip again to remove any leading/trailing spaces created by cleaning
    s = tf.strings.strip(s)

    # Add the <start> and <end> tokens
    s = tf.strings.join(["<start>", s, "<end>"], separator=" ")
    return s

I0000 00:00:1760494975.213441   31154 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13764 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


### Create and Adapt the TextVectorization Layers
With our raw text datasets ready, we need to convert the sentences from strings into integer sequences that our model can understand. In Keras 3, the modern and efficient way to do this is with the `keras.layers.TextVectorization` layer. This layer handles tokenization, vocabulary creation, and integer-encoding directly within our model's graph.

We will create two separate TextVectorization layers: one for the source language (Spanish) and one for the target language (English). We pass our custom_standardization function to the standardize argument to ensure the text is preprocessed according to our specific rules (including adding `<start>` and `<end>` tokens) before tokenization.

The most crucial step is calling the `.adapt()` method. The `adapt` method reads through the text from our training dataset and builds a vocabulary of all the unique words. It's during this step that the layer learns the mapping from each word to a unique integer index. We do this for both the source and target vectorization layers on their respective text data.

***NOTE: This cell takes 10-15 to run while it's adapting the pre-processing layer to our dataset***

In [9]:
# Create and adapt the TextVectorization layers
source_vectorization = keras.layers.TextVectorization(
    max_tokens=None,
    output_mode="int",
    output_sequence_length=None,
    standardize=custom_standardization,
)

target_vectorization = keras.layers.TextVectorization(
    max_tokens=None,
    output_mode="int",
    output_sequence_length=None,
    standardize=custom_standardization,
)

# Adapt the layers to the training data
source_text = train_raw.map(lambda x, y: x)
target_text = train_raw.map(lambda x, y: y)

source_vectorization.adapt(source_text)
target_vectorization.adapt(target_text)

### Create the Final Datasets
Now that our TextVectorization layers have learned the vocabularies, we can build our final input pipelines.

First, we store the vocabulary sizes for later use in our model's Embedding layers. Then, we define two helper functions:

1. `vectorize_text`: This function takes the raw text sentences and applies the appropriate TextVectorization layer to convert them into integer sequences.

2. `create_dataset`: This function prepares the integerized sequences for our encoder-decoder model. The decoder needs two versions of the target sequence during training: one for input (to predict the next word) and one for output (to compare against the prediction for calculating loss). This function creates:
   - target_in: The target sequence with the last token removed.
   - target_out: The target sequence with the first (<start>) token removed.  

Finally, we chain together several `tf.data` methods to construct our final train_dataset and eval_dataset. We `.shuffle()` the training data, `.batch()` both datasets, apply our preprocessing functions using `.map()`, and call `.prefetch()` for better performance. We also inspect the shape of the first batch to confirm our pipeline is set up correctly.

In [10]:
# Set vocabulary sizes
INPUT_VOCAB_SIZE = source_vectorization.vocabulary_size()
TARGET_VOCAB_SIZE = target_vectorization.vocabulary_size()


def vectorize_text(source, target):
    source = source_vectorization(source)
    target = target_vectorization(target)
    return source, target


def create_dataset(source, target):
    target_in = target[:, :-1]
    target_out = target[:, 1:]
    return (source, target_in), target_out


# Create the final training and validation datasets
train_dataset = (
    train_raw.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .map(vectorize_text)
    .map(create_dataset)
    .prefetch(tf.data.AUTOTUNE)
)

eval_dataset = (
    val_raw.batch(BATCH_SIZE)
    .map(vectorize_text)
    .map(create_dataset)
    .prefetch(tf.data.AUTOTUNE)
)

# Set max lengths for the Embedding layers
# We do this by inspecting the element_spec of the dataset
for (source, target_in), target_out in train_dataset.take(1):
    max_length_inp = source.shape[1]
    max_length_targ = target_in.shape[1]
    print("Source shape:", source.shape)
    print("Target in shape:", target_in.shape)
    print("Target out shape:", target_out.shape)

Source shape: (64, 10)
Target in shape: (64, 8)
Target out shape: (64, 8)


### Preview a Batch of Data
To verify that our data pipeline is working correctly, let's take one batch from our train_dataset and inspect the first example. We'll convert the integer tensors back to text to see what the model will receive during training.

In [11]:
# Get the vocabularies from the vectorization layers
source_vocab = source_vectorization.get_vocabulary()
source_index_lookup = {i: word for i, word in enumerate(source_vocab)}

target_vocab = target_vectorization.get_vocabulary()
target_index_lookup = {i: word for i, word in enumerate(target_vocab)}

# Helper function to convert a tensor of token IDs back to a string


def to_text(tensor, lookup_dict):
    # Join the words, filtering out the padding token (ID 0)
    return " ".join([lookup_dict[i] for i in tensor.numpy() if i != 0])


# Take one batch from the training dataset and inspect the first example
for (source_batch, target_in_batch), target_out_batch in train_dataset.take(1):
    # Get the first example from the batch
    source_example = source_batch[0]
    target_in_example = target_in_batch[0]
    target_out_example = target_out_batch[0]

    print("--- Example from a Training Batch ---")
    print("\nSource (Spanish):")
    print(f"  Tensor: {source_example.numpy()}")
    print(f"  Text:   {to_text(source_example, source_index_lookup)}")

    print("\nTarget Input (English - for Decoder):")
    print(f"  Tensor: {target_in_example.numpy()}")
    print(f"  Text:   {to_text(target_in_example, target_index_lookup)}")

    print("\nTarget Output (English - for Loss Calculation):")
    print(f"  Tensor: {target_out_example.numpy()}")
    print(f"  Text:   {to_text(target_out_example, target_index_lookup)}")

--- Example from a Training Batch ---

Source (Spanish):
  Tensor: [ 2 12 45  4  3  0  0  0  0  0]
  Text:   <start> me gusta . <end>

Target Input (English - for Decoder):
  Tensor: [ 2  5 38 10  4  3  0  0]
  Text:   <start> i like it . <end>

Target Output (English - for Loss Calculation):
  Tensor: [ 5 38 10  4  3  0  0  0]
  Text:   i like it . <end>


## Training the RNN encoder-decoder model

We use an encoder-decoder architecture, however we embed our words into a latent space prior to feeding them into the RNN. 

In [12]:
EMBEDDING_DIM = 256
HIDDEN_UNITS = 1024

### Building the Encoder
We will build our translator using a standard encoder-decoder architecture. The encoder's job is to process the entire input sentence and compress its meaning into a fixed-size vector, often called the "thought vector" or "context vector."

We'll use the Keras Functional API to construct the encoder, which gives us a clear way to define the flow of data. Our encoder will have two main layers:

1. **Embedding Layer**: This layer takes the integer-encoded vocabulary and learns a dense vector representation (an embedding) for each word. These embeddings can capture semantic relationships between words.

2. **GRU (Gated Recurrent Unit) Layer**: This is a type of Recurrent Neural Network (RNN) that processes the sequence of word embeddings one by one. We configure it with return_state=True to get the final hidden state of the GRU after it has processed the entire input sentence. This final hidden state is the "thought vector" that encapsulates the meaning of the input sentence and will be passed to the decoder.

In [13]:
encoder_inputs = Input(shape=(None,), name="encoder_input")

encoder_inputs_embedded = Embedding(
    input_dim=INPUT_VOCAB_SIZE,
    output_dim=EMBEDDING_DIM,
    name="encoder_embedding",
)(encoder_inputs)

encoder_rnn = GRU(
    units=HIDDEN_UNITS,
    return_sequences=True,
    return_state=True,
    recurrent_initializer="glorot_uniform",
    name="encoder_gru",
    reset_after=False,
)

encoder_outputs, encoder_state = encoder_rnn(encoder_inputs_embedded)

### Building the Decoder
The decoder takes the "thought vector" from the encoder and generates the translated sentence word by word.

Its architecture mirrors the encoder, containing Embedding and GRU layers. The crucial difference is that we initialize the decoder's GRU layer with the final hidden state of the encoder `(initial_state=encoder_state)`. This is how the decoder receives the context from the source sentence, which it then uses to generate the correct translation.

In [14]:
decoder_inputs = Input(shape=(None,), name="decoder_input")

decoder_inputs_embedded = Embedding(
    input_dim=TARGET_VOCAB_SIZE,
    output_dim=EMBEDDING_DIM,
    name="decoder_embedding",
)(decoder_inputs)

decoder_rnn = GRU(
    units=HIDDEN_UNITS,
    return_sequences=True,
    return_state=True,
    recurrent_initializer="glorot_uniform",
    name="decoder_gru",
    reset_after=False,
)

decoder_outputs, decoder_state = decoder_rnn(
    decoder_inputs_embedded, initial_state=encoder_state
)

The last part of the encoder-decoder architecture is a softmax `Dense` layer that will create the next word probability vector or next word `predictions` from the `decoder_output`:

In [15]:
decoder_dense = Dense(TARGET_VOCAB_SIZE, activation="softmax", name="dense")

predictions = decoder_dense(decoder_outputs)

### Create and Compile the Model
To create the complete trainable model, we instantiate a Model object and specify the encoder and decoder inputs, and the final predictions as the output.

We then compile the model, configuring it for training. We use the `adam` optimizer and the `sparse_categorical_crossentropy` loss function. This loss is ideal for our task because our targets are integers (the word indices) and the model's output is a probability distribution over the vocabulary.

Finally, we call `.summary()` to print a useful overview of the model's architecture, including the layers, output shapes, and the number of trainable parameters.

In [16]:
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=predictions)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 256) │  1,970,688 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 256) │  1,079,552 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_gru (GRU)   │ [(None, None,     │  3,935,232 │ encoder_embeddin… │
│                     │ 1024), (None,     │            │                   │
│                     │ 1024)]            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_gru (GRU)   │ [(None, None,     │  3,935,232 │ decoder_embeddin… │
│                     │ 1024), (None,     │            │ encoder_gru[0][1] │
│                     │ 1024)]            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  4,322,425 │ decoder_gru[0][0] │
│                     │ 4217)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,243,129 (58.15 MB)

 Trainable params: 15,243,129 (58.15 MB)

 Non-trainable params: 0 (0.00 B)

Let's now train the model!

***NOTE: Update the number of `EPOCHS` to 10/15 to get a decent translation performance. However, this will increase the training time.***

In [17]:
EPOCHS = 5

history = model.fit(
    train_dataset, validation_data=eval_dataset, epochs=EPOCHS, verbose=2
)

Epoch 1/5


I0000 00:00:1760495324.894985   31190 service.cc:148] XLA service 0x7f6fd400d6c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760495324.895027   31190 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
W0000 00:00:1760495325.163860   31190 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1760495325.322773   31190 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1760495328.054608   31190 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1760495328.328663   31191 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1760495331.116997   31190 assert_op.cc:38] Ignoring Assert operator compile_loss/spar

375/375 - 67s - 178ms/step - loss: 2.7221 - val_loss: 2.7872
Epoch 2/5
375/375 - 20s - 54ms/step - loss: 1.7850 - val_loss: 2.3783
Epoch 3/5
375/375 - 18s - 49ms/step - loss: 1.2807 - val_loss: 2.1386
Epoch 4/5
375/375 - 17s - 44ms/step - loss: 0.8743 - val_loss: 2.0280
Epoch 5/5
375/375 - 16s - 43ms/step - loss: 0.5724 - val_loss: 2.0372


### Implementing the Translation (Inference) Function

Now that our model is trained, we need a way to use it for translation. We can't simply call `model.predict()` on a Spanish sentence because the model expects both a source (Spanish) and a target (English) sentence as input. During training, we used the ground-truth target sentence in a technique called "teacher forcing." For inference, we don't have the target sentence—that's what we need to generate!

The solution is to generate the translation word by word in a loop. The process works like this:

1.  Take the input sentence (e.g., "No estamos comiendo.") and pass it through the encoder to get its final hidden state (the "thought vector").
2.  Start the decoder with the `<start>` token.
3.  Feed the `<start>` token and the encoder's state into the decoder to predict the first word of the translation.
4.  Take the predicted word, feed it back into the decoder as input for the next step, and use the new decoder state.
5.  Repeat this process until the decoder predicts the `<end>` token, signaling that the translation is complete.

To implement this, we will create two new, specialized models using the layers from the model we just trained:

*   An **`encoder_model`** that takes a raw string sentence, vectorizes it, and returns the encoder's final hidden state.
*   A **`decoder_model`** that takes the current predicted sequence and a hidden state, and returns the prediction for the next word, along with the updated hidden state.

The following cells will define these two models and the `decode_sequences` function that uses them to perform the translation.


In [18]:
# --- Create Inference Encoder Model ---
# This model will convert raw Spanish sentences into the encoder's final state.

# Input layer for raw strings
encoder_string_input = keras.Input(
    shape=(1,), dtype="string", name="encoder_string_input"
)

# Vectorize the strings using the adapted layer
encoder_vectorized = source_vectorization(encoder_string_input)

# Reuse the trained Embedding layer
encoder_embedding_layer = model.get_layer("encoder_embedding")
encoder_embedded = encoder_embedding_layer(encoder_vectorized)

# Reuse the trained GRU layer
encoder_gru_layer = model.get_layer("encoder_gru")
_, encoder_state_output = encoder_gru_layer(encoder_embedded)

# Create the final encoder model for inference
encoder_model = keras.Model(
    inputs=encoder_string_input,
    outputs=encoder_state_output,
    name="inference_encoder",
)

In [19]:
# --- Create Inference Decoder Model ---
# This model's structure is similar to the original, as it works with token IDs.

# Input layers for the decoder
decoder_input = keras.Input(shape=(None,), name="decoder_input")
decoder_state_input = keras.Input(
    shape=(HIDDEN_UNITS,), name="decoder_state_input"
)

# Reuse trained layers from the main model
decoder_embedding_layer = model.get_layer("decoder_embedding")
decoder_gru_layer = model.get_layer("decoder_gru")
decoder_dense_layer = model.get_layer("dense")

# Build the decoder graph
decoder_embedded = decoder_embedding_layer(decoder_input)
decoder_outputs, decoder_state_output = decoder_gru_layer(
    decoder_embedded, initial_state=decoder_state_input
)
decoder_predictions = decoder_dense_layer(decoder_outputs)

# Create the final decoder model for inference
decoder_model = keras.Model(
    inputs=[decoder_input, decoder_state_input],
    outputs=[decoder_predictions, decoder_state_output],
    name="inference_decoder",
)

### Performing the Translation

With our specialized encoder and decoder models ready, we can now define the main `decode_sequences` function that ties everything together to perform the translation.

This function implements the step-by-step decoding process:

1.  **Encode the input**: The raw input sentences are passed to the `encoder_model` to get the initial "thought vector" or hidden state.

2.  **Initialize the sequence**: A target sequence is created, containing only the `<start>` token for each sentence in the batch.

3.  **Iteratively decode**: In a loop, the `decoder_model` is called with the current target sequence and the hidden state to predict the next token. For simplicity, we use `argmax` to select the most probable token as our prediction.

4.  **Append and update**: The predicted token is appended to our result, and the process is repeated with the new token and the updated hidden state from the decoder.

5.  **Stop condition**: The loop continues until the special `<end>` token is generated or the maximum sequence length is reached.

Finally, we'll test our function with a few example sentences to see the translation results in action.


In [20]:
# Get the vocabulary and start/end token IDs from the adapted TextVectorization layer
targ_vocab = target_vectorization.get_vocabulary()
targ_index_lookup = dict(zip(range(len(targ_vocab)), targ_vocab))
start_token_id = target_vectorization.get_vocabulary().index("<start>")
end_token_id = target_vectorization.get_vocabulary().index("<end>")


def decode_sequences(input_sentences, max_decode_length=50):
    """
    Arguments:
        input_sentences: A list of raw strings in the source language.
    Returns:
        A list of translated sentences.
    """
    batch_size = tf.shape(input_sentences)[0]

    # Encode the input strings to get the initial state.
    states_value = encoder_model(input_sentences)

    # Initialize the target sequence with the <start> token ID.
    target_seq = tf.fill([batch_size, 1], start_token_id)
    decoded_sentences = ["" for _ in range(batch_size)]

    for i in range(max_decode_length):
        output_tokens, decoder_state = decoder_model([target_seq, states_value])

        # Sample a token (we use argmax for simplicity)
        sampled_token_index = tf.argmax(output_tokens[:, -1, :], axis=-1)

        for j in range(batch_size.numpy()):
            token = targ_index_lookup[sampled_token_index[j].numpy()]
            if token == "<end>":
                continue  # Don't add the end token to the output
            decoded_sentences[j] += token + " "

        # Update the target sequence for the next iteration
        target_seq = tf.expand_dims(sampled_token_index, axis=-1)

        # Update the decoder state
        states_value = decoder_state

    return [s.strip() for s in decoded_sentences]


# Example usage:
sentences = [
    "No estamos comiendo.",
    "Está llegando el invierno.",
]

machine_translations = decode_sequences(tf.constant(sentences))

Now we're ready to predict!

In [21]:
sentences = [
    "No estamos comiendo.",
    "Está llegando el invierno.",
    "El invierno se acerca.",
    "Tom no comio nada.",
    "Su pierna mala le impidió ganar la carrera.",
    "Su respuesta es erronea.",
    "¿Qué tal si damos un paseo después del almuerzo?",
]

reference_translations = [
    "We're not eating.",
    "Winter is coming.",
    "Winter is coming.",
    "Tom ate nothing.",
    "His bad leg prevented him from winning the race.",
    "Your answer is wrong.",
    "How about going for a walk after lunch?",
]

machine_translations = decode_sequences(tf.constant(sentences))

for i in range(len(sentences)):
    print("-")
    print("INPUT:")
    print(sentences[i])
    print("REFERENCE TRANSLATION:")
    print(reference_translations[i])
    print("MACHINE TRANSLATION:")
    print(machine_translations[i])

-
INPUT:
No estamos comiendo.
REFERENCE TRANSLATION:
We're not eating.
MACHINE TRANSLATION:
we re not eating .
-
INPUT:
Está llegando el invierno.
REFERENCE TRANSLATION:
Winter is coming.
MACHINE TRANSLATION:
they re doing worse .
-
INPUT:
El invierno se acerca.
REFERENCE TRANSLATION:
Winter is coming.
MACHINE TRANSLATION:
the worst is over .
-
INPUT:
Tom no comio nada.
REFERENCE TRANSLATION:
Tom ate nothing.
MACHINE TRANSLATION:
tom won t answer .
-
INPUT:
Su pierna mala le impidió ganar la carrera.
REFERENCE TRANSLATION:
His bad leg prevented him from winning the race.
MACHINE TRANSLATION:
his novel sold well .
-
INPUT:
Su respuesta es erronea.
REFERENCE TRANSLATION:
Your answer is wrong.
MACHINE TRANSLATION:
the pen is broken .
-
INPUT:
¿Qué tal si damos un paseo después del almuerzo?
REFERENCE TRANSLATION:
How about going for a walk after lunch?
MACHINE TRANSLATION:
how about i m here ?


## Evaluation Metric (BLEU)

Unlike say, image classification, there is no one right answer for a machine translation. However our current loss metric, cross entropy, only gives credit when the machine translation matches the exact same word in the same order as the reference translation. 

Many attempts have been made to develop a better metric for natural language evaluation. The most popular currently is Bilingual Evaluation Understudy (BLEU).

- It is quick and inexpensive to calculate.
- It allows flexibility for the ordering of words and phrases.
- It is easy to understand.
- It is language independent.
- It correlates highly with human evaluation.
- It has been widely adopted.

The score is from 0 to 1, where 1 is an exact match.

It works by counting matching n-grams between the machine and reference texts, regardless of order. BLUE-4 counts matching n grams from 1-4 (1-gram, 2-gram, 3-gram and 4-gram). It is common to report both BLUE-1 and BLUE-4

It still is imperfect, since it gives no credit to synonyms and so human evaluation is still best when feasible. However BLEU is commonly considered the best among bad options for an automated metric.

The Hugging Face evaluate framework has an implementation that we will use.

We can't run calculate BLEU during training, because at that time the correct decoder input is used. Instead we'll calculate it now.

For more info: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

In [22]:
def postprocess(sentence):
    filtered = list(filter(lambda x: x != "" and x != "<end>", sentence))
    return " ".join(filtered)

Let's now average the `bleu_1` and `bleu_4` scores for all the sentence pairs in the eval set.

In [23]:
NUM_EVALUATE = 1000

source_sentences = []
reference = []  # This will be a list of lists for the BLEU score calculation

# Iterate over the raw validation dataset to get the source and reference texts.
# We use .numpy().decode() to convert the EagerTensors from the dataset into strings.
for spa, eng in tqdm(val_raw.take(NUM_EVALUATE), total=NUM_EVALUATE):
    source_sentences.append(spa.numpy().decode("utf-8"))
    reference.append([eng.numpy().decode("utf-8")])

# Generate all machine translations in a single batch for better performance.
candidate = decode_sequences(tf.constant(source_sentences))

# Print a few examples to see the results
for i in range(5):
    print("-" * 20)
    print(f"Source:    {source_sentences[i]}")
    print(f"Reference: {reference[i][0]}")
    print(f"Machine:   {candidate[i]}")

100%|██████████| 1000/1000 [00:00<00:00, 3739.85it/s]


--------------------
Source:    No lo dije en serio.
Reference: I meant it as a joke.
Machine:   i didn t say that .
--------------------
Source:    Ayer me encontré con Mary.
Reference: I met Mary yesterday.
Machine:   i saw her tie mary .
--------------------
Source:    Me encontré con Tom después del trabajo.
Reference: I met Tom after work.
Machine:   i saw tom on mary .
--------------------
Source:    Encontré a Tom después del trabajo.
Reference: I met Tom after work.
Machine:   i saw tom on monday .
--------------------
Source:    Me encontré en el camino con Tom.
Reference: I met Tom on the way.
Machine:   i saw tom on tv .


### Check the score

In [24]:
bleu = evaluate.load("bleu")
bleu_1 = bleu.compute(predictions=candidate, references=reference, max_order=1)
bleu_4 = bleu.compute(predictions=candidate, references=reference, max_order=4)

In [25]:
bleu_1["bleu"]

0.3737112580575483

In [26]:
bleu_4["bleu"]

0.06999233485937993

Copyright 2025 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License